In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [20]:
from multiprocessing import Pool, Value, Manager
from multiprocessing import Queue as PQueue
import multiprocessing
import numpy as np
import pandas as pd
import time
import sys


class Parallel(object):

    """
    generate samples from some data structure
    """

    def __init__(self, objective, objective_kwargs):
        self.objective = objective
        self.objective_kwargs = objective_kwargs
        return

    def run(self, iterable_obj, n_jobs=-1):

        try:
            iter(iterable_obj)
        except Exception as e:
            raise Exception(e.__str__())

        self.X = Manager().list()
        if n_jobs == -1:
            import multiprocessing
            n_jobs = multiprocessing.cpu_count()
        pool = Pool(n_jobs)

        params = []
        for item in iterable_obj:
            params.append((item,))
        pool.starmap(self.worker, params)
        pool.close()
        pool.join()
        return list(self.X)


    def worker(self, item):
        try:
            d_ = self.objective(item, **self.objective_kwargs)
            assert type(d_)==dict, 'objective not return a dict object'
            self.X.append({**d_})
        except Exception as e:
            raise Exception()


"\n\nlst_ = [1,2,3,4]\n\ndef objective(item):\n    return {'data':item + 3}\n\nparalel = Parallel(objective)\n\ndf_ = pd.DataFrame(paralel.run(lst_))\n\n"

In [28]:
a = [1,2,3,4,5,6,7,8,9,10]

In [42]:
df_info = pd.DataFrame({'a':[1,2,3], 'b':[5,6,7]})

def func_b_sub(a, b, c, df):
    return a + b + c + df.shape[0]*10

def func_b(item,  cc, **kwargs):
    id_ = item[0]
    x_ = item[1]
    return {'b':x_ + 10 + func_b_sub(**kwargs), 'idx':id_}

In [43]:
r = []
for item in enumerate(a):
    b = func_b(item, 10, a=100,b=100,c=100, df=df_info)
    r.append(b)
print(r)

[{'b': 341, 'idx': 0}, {'b': 342, 'idx': 1}, {'b': 343, 'idx': 2}, {'b': 344, 'idx': 3}, {'b': 345, 'idx': 4}, {'b': 346, 'idx': 5}, {'b': 347, 'idx': 6}, {'b': 348, 'idx': 7}, {'b': 349, 'idx': 8}, {'b': 350, 'idx': 9}]


In [45]:
r = Parallel(func_b,{'cc':10,'a':100,'b':100,'c':100, 'df':df_info}).run(enumerate(a))
print(r)

[{'b': 344, 'idx': 3}, {'b': 343, 'idx': 2}, {'b': 341, 'idx': 0}, {'b': 342, 'idx': 1}, {'b': 348, 'idx': 7}, {'b': 346, 'idx': 5}, {'b': 345, 'idx': 4}, {'b': 349, 'idx': 8}, {'b': 350, 'idx': 9}, {'b': 347, 'idx': 6}]


In [6]:
def parallel_decorator(func):
    def function_wrapper(**kwargs):
        func(**kwargs)
    return function_wrapper

@our_decorator
def foo(x, y, z):
    print("Hi, foo has been called with " + str(x))

foo("Hi", 'b', 'c')

Before calling foo
Hi, foo has been called with Hi
After calling foo


In [ ]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

from decorators import do_twice

@do_twice
def say_whee():
    print("Whee!")